# **Colab Code**

In [ ]:
# ==========================================================
# 1) INSTALL DEPENDENCIES
# ==========================================================
!pip install -q ultralytics roboflow

import os
import random
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from ultralytics import YOLO
from roboflow import Roboflow


# ==========================================================
# 2) DOWNLOAD DATASET FROM ROBOFLOW (METHOD 1)
# ==========================================================
ROBOFLOW_API_KEY = "PASTE_YOUR_API_KEY_HERE"

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("yolox-qcftu").project("indian-number-plate-keeo5")
dataset = project.version(2).download("yolov8")

print("✅ Dataset downloaded successfully!")
print("Dataset location:", dataset.location)


# ==========================================================
# 3) CHECK DATASET STRUCTURE
# ==========================================================
!ls {dataset.location}


# ==========================================================
# 4) FIX data.yaml (IMPORTANT STEP)
# Roboflow gives ../train/images which fails in Colab
# ==========================================================
# FIX data.yaml (IMPORTANT STEP)
# Roboflow gives ../train/images which fails in Colab

# FIXED_YAML_PATH = "/content/data.yaml"

#FIXED_YAML_PATH = "/content/Indian-number-plate-2/data_fixed.yaml"

FIXED_YAML_PATH = f"{dataset.location}/data_fixed.yaml"

yaml_text = f"""
train: {dataset.location}/train/images
val: {dataset.location}/valid/images
test: {dataset.location}/test/images

nc: 1
names: ['number_plate']
"""

with open(FIXED_YAML_PATH, "w") as f:
    f.write(yaml_text)

print("✅ Fixed YAML created at:", FIXED_YAML_PATH)


# ==========================================================
# 5) VISUALIZE RANDOM TRAIN IMAGES (BEFORE TRAINING)
# ==========================================================
TRAIN_IMG_DIR = f"{dataset.location}/train/images"

train_images = os.listdir(TRAIN_IMG_DIR)
sample_imgs = random.sample(train_images, 9)

plt.figure(figsize=(12, 12))

for i, img_name in enumerate(sample_imgs):
    img_path = os.path.join(TRAIN_IMG_DIR, img_name)

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.subplot(3, 3, i+1)
    plt.imshow(img)
    plt.title(img_name, fontsize=8)
    plt.axis("off")

plt.tight_layout()
plt.show()


# ==========================================================
# 6) TRAIN YOLO MODEL
# ==========================================================
model = YOLO("yolov8n.pt")

model.train(
    data=FIXED_YAML_PATH,
    epochs=50,
    imgsz=640,
    batch=16
)


# ==========================================================
# 7) PLOT TRAINING GRAPH (mAP50 over Epochs)
# ==========================================================
results_csv = "/content/runs/detect/train/results.csv"
df = pd.read_csv(results_csv)

plt.figure(figsize=(10, 5))
plt.plot(df["epoch"], df["metrics/mAP50(B)"])
plt.title("mAP50 over Epochs")
plt.xlabel("Epoch")
plt.ylabel("mAP50")
plt.grid(True)
plt.show()


# ==========================================================
# 8) LOAD BEST MODEL
# ==========================================================
BEST_MODEL_PATH = "/content/runs/detect/train/weights/best.pt"
best_model = YOLO(BEST_MODEL_PATH)


# ==========================================================
# 9) EVALUATE ON TEST DATASET (BEFORE PREDICTION)
# ==========================================================
best_model.val(
    data=FIXED_YAML_PATH,
    split="test"
)


# ==========================================================
# 10) VISUALIZE RANDOM TEST PREDICTIONS (AFTER TRAINING)
# ==========================================================
TEST_IMG_DIR = f"{dataset.location}/test/images"

test_images = os.listdir(TEST_IMG_DIR)
sample_test_imgs = random.sample(test_images, 9)

plt.figure(figsize=(12, 12))

for i, img_name in enumerate(sample_test_imgs):
    img_path = os.path.join(TEST_IMG_DIR, img_name)

    results = best_model.predict(img_path, conf=0.4, verbose=False)[0]

    annotated_img = results.plot()
    annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

    plt.subplot(3, 3, i+1)
    plt.imshow(annotated_img)
    plt.title(img_name, fontsize=8)
    plt.axis("off")

plt.tight_layout()
plt.show()


# ==========================================================
# 11) DOWNLOAD BEST MODEL (best.pt)
# ==========================================================
from google.colab import files
files.download(BEST_MODEL_PATH)


In [ ]:
# [You can use path: dataset in yaml like this
# path: dataset

# train: train/images
# val: valid/images
# test: test/images

# nc: 1
# names: ['number_plate']

# roboflow:
#   workspace: yolox-qcftu
#   project: indian-number-plate-keeo5
#   version: 2
#   license: CC BY 4.0
#   url: https://universe.roboflow.com/yolox-qcftu/indian-number-plate-keeo5/dataset/2]


# **Kaggle Code**

In [ ]:
# ==========================================================
# 1) INSTALL ULTRALYTICS
# ==========================================================
!pip install -q ultralytics

import os
import random
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import shutil
from ultralytics import YOLO


# ==========================================================
# 2) VISUALIZE RANDOM TRAIN IMAGES (BEFORE TRAINING)
# ==========================================================
TRAIN_IMG_DIR = "/kaggle/input/cardetection/car/train/images"

train_images = os.listdir(TRAIN_IMG_DIR)
sample_imgs = random.sample(train_images, 9)

plt.figure(figsize=(12, 12))

for i, img_name in enumerate(sample_imgs):
    img_path = os.path.join(TRAIN_IMG_DIR, img_name)

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.subplot(3, 3, i+1)
    plt.imshow(img)
    plt.title(img_name, fontsize=8)
    plt.axis("off")

plt.tight_layout()
plt.show()


# ==========================================================
# 3) TRAIN YOLO MODEL
# ==========================================================
model = YOLO("yolov8n.pt")

model.train(
    data="/kaggle/input/cardetection/car/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16
)


# ==========================================================
# 4) PLOT TRAINING GRAPH (mAP50 over Epochs)
# ==========================================================
results_csv = "/kaggle/working/runs/detect/train/results.csv"
df = pd.read_csv(results_csv)

plt.figure(figsize=(10, 5))
plt.plot(df["epoch"], df["metrics/mAP50(B)"])
plt.title("mAP50 over Epochs")
plt.xlabel("Epoch")
plt.ylabel("mAP50")
plt.grid(True)
plt.show()


# ==========================================================
# 5) LOAD BEST MODEL
# ==========================================================
BEST_MODEL_PATH = "/kaggle/working/runs/detect/train/weights/best.pt"
best_model = YOLO(BEST_MODEL_PATH)


# ==========================================================
# 6) EVALUATE ON TEST DATASET (BEFORE PREDICTION)
# ==========================================================
best_model.val(
    data="/kaggle/input/cardetection/car/data.yaml",
    split="test"
)


# ==========================================================
# 7) VISUALIZE RANDOM TEST PREDICTIONS (AFTER TRAINING)
# ==========================================================
TEST_IMG_DIR = "/kaggle/input/cardetection/car/test/images"

test_images = os.listdir(TEST_IMG_DIR)
sample_test_imgs = random.sample(test_images, 9)

plt.figure(figsize=(12, 12))

for i, img_name in enumerate(sample_test_imgs):
    img_path = os.path.join(TEST_IMG_DIR, img_name)

    results = best_model.predict(img_path, conf=0.4, verbose=False)[0]

    annotated_img = results.plot()
    annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

    plt.subplot(3, 3, i+1)
    plt.imshow(annotated_img)
    plt.title(img_name, fontsize=8)
    plt.axis("off")

plt.tight_layout()
plt.show()


# ==========================================================
# 8) PYTHONIC COPY BEST MODEL TO OUTPUT FOLDER (FOR DOWNLOAD)
# ==========================================================
src = "/kaggle/working/runs/detect/train/weights/best.pt"
dst = "/kaggle/working/best.pt"

shutil.copy(src, dst)

print("✅ best.pt copied to:", dst)
print("📌 Now go to Kaggle Output tab and download best.pt")


In [ ]:
# split = test means test folder?

# Yes ✅

# split="test" means YOLO will use the test folder defined in your data.yaml.

# Example data.yaml:

# train: train/images
# val: valid/images
# test: test/images


# So:

# Default (no split given)
# model.val(data="data.yaml")


# ➡️ uses val/images

# With split="test"
# model.val(data="data.yaml", split="test")


# ➡️ uses test/images

# So yes, split="test" = evaluate on test folder.